In [1]:
import statsapi
import psycopg2
from baseball_data import generateGamePksFromDates
import psycopg2.extras
import time
from api2psql import *

In [3]:
conn = psycopg2.connect(
    host = "localhost",
    database = "baseball",
    user = "karisch",
    password = "cocacola",
    port = 5432
)
#conn.autocommit = True

In [5]:
#conn.commit()
conn.close()

In [4]:
populateTeamTable(conn)

In [11]:
jack = statsapi.get('league', {'sportId': 1})
jack['leagues'][0]

{'id': 103,
 'name': 'American League',
 'link': '/api/v1/league/103',
 'abbreviation': 'AL',
 'nameShort': 'American',
 'seasonState': 'postseason',
 'hasWildCard': True,
 'hasSplitSeason': False,
 'numGames': 60,
 'hasPlayoffPoints': False,
 'numTeams': 15,
 'numWildcardTeams': 2,
 'seasonDateInfo': {'regularSeasonStartDate': '2020-07-23',
  'regularSeasonEndDate': '2020-09-27',
  'preSeasonStartDate': '2020-02-21',
  'preSeasonEndDate': '2020-07-22',
  'postSeasonStartDate': '2020-09-29',
  'postSeasonEndDate': '2020-10-28',
  'lastDate1stHalf': '2020-08-25',
  'firstDate2ndHalf': '2020-08-26'},
 'season': '2020',
 'orgCode': 'AL',
 'conferencesInUse': False,
 'divisionsInUse': True,
 'sport': {'id': 1, 'link': '/api/v1/sports/1'},
 'sortOrder': 21}

In [9]:
sports = statsapi.get('sports',{})['sports']

args = []
for sport in sports:
    leagues = statsapi.get('league', {'sportId': sport})['leagues']
    for league in leagues:
        args.append(
            (
                dictTry(league, ['id']),
                dictTry(league, ['name']),
                dictTry(league, ['season']),
            )
        )

In [28]:
jack = statsapi.get('sports', {})

In [29]:
jack['sports']

[{'id': 1,
  'code': 'mlb',
  'link': '/api/v1/sports/1',
  'name': 'Major League Baseball',
  'abbreviation': 'MLB',
  'sortOrder': 11,
  'activeStatus': True},
 {'id': 11,
  'code': 'aaa',
  'link': '/api/v1/sports/11',
  'name': 'Triple-A',
  'abbreviation': 'AAA',
  'sortOrder': 101,
  'activeStatus': True},
 {'id': 12,
  'code': 'aax',
  'link': '/api/v1/sports/12',
  'name': 'Double-A',
  'abbreviation': 'AA',
  'sortOrder': 301,
  'activeStatus': True},
 {'id': 13,
  'code': 'afa',
  'link': '/api/v1/sports/13',
  'name': 'Class A Advanced',
  'abbreviation': 'A(Adv)',
  'sortOrder': 501,
  'activeStatus': True},
 {'id': 14,
  'code': 'afx',
  'link': '/api/v1/sports/14',
  'name': 'Class A',
  'abbreviation': 'A(Full)',
  'sortOrder': 701,
  'activeStatus': True},
 {'id': 15,
  'code': 'asx',
  'link': '/api/v1/sports/15',
  'name': 'Class A Short Season',
  'abbreviation': 'A(Short)',
  'sortOrder': 901,
  'activeStatus': True},
 {'id': 5442,
  'code': 'roa',
  'link': '/api/v